# Scenario 3: Multiple data scientists working on multiple ML models

MLflow setup:

- Tracking server: yes, remote server (GCP Compute Engine `e2-standard-2`).
- Backend store: `postgresql` database.
- Artifacts store: Google Storage Bucket.

The experiments can be explored by accessing the remote server.

The example uses GCP to host a remote server. In order to run the example you'll need a GCP account. Follow the steps described in the [MLFlow GCP Guide](../../notes/mlflow_gcp.md) to set up the tracking server.

In [1]:
import mlflow
import os

TRACKING_SERVER_HOST = "34.64.84.237" # fill in with the public DNS of the EC2 instance
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:5000")

service_account_key_path = "../../../service_account_key.json"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = service_account_key_path


In [2]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")


tracking URI: 'http://34.64.84.237:5000'


In [3]:
mlflow.search_experiments()

[<Experiment: artifact_location='gs://mlflow-artifacts-bucket-1/mlruns/0', creation_time=1703340769879, experiment_id='0', last_update_time=1703340769879, lifecycle_stage='active', name='Default', tags={}>]

In [4]:
mlflow.get_artifact_uri()

'gs://mlflow-artifacts-bucket-1/mlruns/0/cebd9f8b0bb74bdeab7277917bd64f24/artifacts'

In [5]:
mlflow.end_run()

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2023/12/23 14:13:24 INFO mlflow.tracking.fluent: Experiment with name 'my-experiment-1' does not exist. Creating a new experiment.


default artifacts URI: 'gs://mlflow-artifacts-bucket-1/mlruns/1/4626248fb7714d80a12d6ffd1a021976/artifacts'


## Interacting with the model registry

In [7]:
from mlflow.tracking import MlflowClient


client = MlflowClient(f"http://{TRACKING_SERVER_HOST}:5000")

In [8]:
client.search_registered_models()

[]

In [9]:
runs = client.search_runs(
    experiment_ids="1",
    max_results=3,
)

In [10]:
run = runs[0]
run_id = run.info.run_id
print(run_id)

4626248fb7714d80a12d6ffd1a021976


In [11]:
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier'
)

Successfully registered model 'iris-classifier'.
2023/12/23 14:13:27 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classifier, version 1
Created version '1' of model 'iris-classifier'.


<ModelVersion: aliases=[], creation_timestamp=1703340807797, current_stage='None', description='', last_updated_timestamp=1703340807797, name='iris-classifier', run_id='4626248fb7714d80a12d6ffd1a021976', run_link='', source='gs://mlflow-artifacts-bucket-1/mlruns/1/4626248fb7714d80a12d6ffd1a021976/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='1'>